In [ ]:
%load_ext autoreload

%autoreload 2
import gcsfs
import numpy as np
import zarr
fs = gcsfs.GCSFileSystem()


In [ ]:
from oxeo.water.models.utils import load_tile, TilePath, tile_from_id, resize_sample
from oxeo.water.models.segmentation import Segmentation2DPredictor
from oxeo.water.models.segmentation.segmentation_2d import reconstruct_from_patches


tile_path = TilePath(tile_from_id("43_P_10000_65_137"), "landsat-8")

seg_predictor = Segmentation2DPredictor(
    batch_size=16,
    ckpt_path="gs://oxeo-models/semseg/epoch_011.ckpt",
    input_channels=6,
    num_classes=3,
    chip_size=250,
    fs=fs,
)


In [ ]:
tile = load_tile(
    fs.get_mapper,
    tile_path,
    revisit=slice(4, 6),
    bands=["nir", "red", "green", "blue", "swir1", "swir2"],
)


In [ ]:
res = seg_predictor.predict(tile, target_size=1000)


In [ ]:
res.shape


In [ ]:
import matplotlib.pyplot as plt
from skimage.exposure import equalize_adapthist, rescale_intensity

i = 1
plt.imshow(res[i])
plt.show()
img = tile["image"].numpy()[i][[1, 2, 3]].transpose(1, 2, 0)
vmin, vmax = np.percentile(img, q=(2, 98))
img = rescale_intensity(img, in_range=(vmin, vmax), out_range=(0, 1))
plt.imshow(img)
